# Erstellung des Datensatzes 
Texte: Blogbeiträge der Blogplattform Hypotheses.org
Labels: Die von den Wissenschaftlern gewählten Themen und Disziplinen

Autorin: Maria Hartmann

In [2]:
# Import libraries
import numpy as np
import csv # for csv output
import requests # HTTP for humans
from bs4 import BeautifulSoup # module for web scraping
import _thread 
from threading import Thread # to start parallel threads
import time # to get the processing time
import os
import shutil # to move files
from collections import Counter # to count element appearences in a list

Einlesen der Ausgangsdatei metadata.csv

In [4]:
# metadata.csv einlesen
folder = '../Preprocessing'
file = folder+'/metadata.csv'
lines = [] # all lines from metadata
de_lines = [] # german lines from metadata

with open(file, 'r', encoding='utf-8') as openfile:
    metadata = openfile.readlines()
    openfile.close()
    for i, line in enumerate(metadata):
        lines.append(line.replace('\n', '').split(";"))
        if lines[i][1] == "de":
            de_lines.append(lines[i])
        else:
            continue

    
# de_lines in numpy_array umgewandelt, weil Zugriff schneller geht, kann aber nicht verändert werden
np_lines = np.array(de_lines)
print(type(np_lines))

# Blogs ohne Disziplinen aus metadata.csv rausfiltern und in error_lines.csv schreiben
# Fehlerhafte Blogs (z.B. nicht mehr verfügbar oder einmalig andere Sprache) übergehen (wegschmeißen)
# die restlichen deutschen Daten in de_lines.csv schreiben
with open(folder+'/de_lines.csv', 'w', newline='', encoding="utf-8") as decsv, open(folder+'/unlabeled_lines.csv', 'w', newline='', encoding="utf-8") as unlabeledcsv, open(folder+'/error_lines.csv', 'w', newline='', encoding="utf-8") as errorcsv: 
    de = csv.writer(decsv, delimiter = ";")
    unlabeled = csv.writer(unlabeledcsv, delimiter = ";")
    errors = csv.writer(errorcsv, delimiter = ";")
    for i, line in enumerate(np_lines):
        if (np_lines[i][7] == "marginalie") or (np_lines[i][7] == "ciera"):
            # keine Disziplinen zugeordnet, 
            unlabeled.writerow(line)
        elif (np_lines[i][7] == "holocaustwebsites"):
        # holocaustwebsites rausgefiltert, weil diese Website  nicht mehr verfügbar ist
        # alles andere wird über den Blogpost-per-Blog-Index gefiltert
        #elif (np_lines[i][7] == "holocaustwebsites") or (np_lines[i][7] == "aleesp") or (np_lines[i][7] == "filstoria") or (np_lines[i][7] == "atb"):
            # aleesp rausgefiltert, weil es eigentlich ein spanischer Blog ist und im deutschen Korpus nur 1x vorkommt
            # filstoria rausgefiltert, weil es eigentlich ein italienischer Blog ist und im deutschen Korpus nur 1x vorkommt
            # atb rausgefiltert, weil Disciplines und Themes fehlerhaft sind (mit Doppelpunkt) und der Blog mehrheitlich englisch ist
            errors.writerow(line)
        else:
            de.writerow(line)


# de_lines.csv in data einlesen, um die Fehler nicht mit einlesen zu müssen 
data = [] # alle lines aus de_lines, ohne errors
bloglist = [] # alle blogs, die in de_lines vorkommen
with open(folder+'/de_lines.csv', 'r', encoding='utf-8') as openfile:
    de_csv = openfile.readlines()
    openfile.close()
    for i, line in enumerate(de_csv):
        data.append(line.replace('\n', '').split(";")) 
        bloglist.append(data[i][7])




<class 'numpy.ndarray'>


In [5]:
# remove blogs with less than 10 posts, damit anderssprachige blogs mit einzelnen deutschen Posts rausgefiltert werden

c = Counter(bloglist)
blog_select = []
counter = 0
for key in sorted(c): 
    if c[key] < 10:
        #print("%s: %s" % (key, c[key]))
        blog_select.append(key)
        counter += c[key]


trainset = [x for x in data if x[7] not in blog_select]


print(len(data))
print(len(trainset))
print(len(bloglist))
print(len(blog_select))
print(counter)

21695
21387
21695
119
308


Auslesen der Klassenbezeichnungen 

In [ ]:
# crawl subjects and themes
errorlist = []
def get_disciplines(index):
    #print("\nline 1:", line)
    #print(i+1)
    #print(line)
    url = trainset[index][9]
    #print(i, "\npage:", url)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")

    if(soup.find(title="Zum Hypotheses-Katalogeintrag")):
        element = soup.find(title="Zum Hypotheses-Katalogeintrag")
        link = element.get("href")
        #print("link:", link)
        
    elif(soup.find(title="Zum OpenEdition-Katalogeintrag")):
        element = soup.find(title="Zum OpenEdition-Katalogeintrag")
        link = element.get("href")
        #print("link:", link)

    elif(soup.find(title="Ce carnet dans le catalogue d'Hypothèses")):
        element = soup.find(title="Ce carnet dans le catalogue d'Hypothèses")
        link = element.get("href")
        #print("link:", link)
        
    elif(soup.find(title="Ce carnet dans le catalogue d'OpenEdition")):
        element = soup.find(title="Ce carnet dans le catalogue d'OpenEdition")
        link = element.get("href")
        #print("link:", link)    

    elif(soup.find(title="This blog in Hypotheses catalogue")):
        element = soup.find(title="This blog in Hypotheses catalogue")
        link = element.get("href")
        #print("link:", link)
        
    elif(soup.find(title="This blog in OpenEdition catalogue")):
        element = soup.find(title="This blog in OpenEdition catalogue")
        link = element.get("href")
        #print("link:", link)

    else:
        print("Kein Open-Edition-Link gefunden!", index, trainset[index])
        trainset[index].append("Kein Open-Edition-Link gefunden!")
        errorlist.append(line)
        return

    subpage = requests.get(link)
    #print(subpage)
    subsoup = BeautifulSoup(subpage.text, "html.parser")
    morelinks = subsoup.find(class_="more-links")
    disciplines = []
    for i, child in enumerate(morelinks.children):
        #print("disciplines:", i, child)
        disciplines.append(child)

    #print(disciplines[9])
    #print(disciplines[14])
    if len(disciplines) > 13:
        trainset[index].append(disciplines[9].replace("\n", "").strip())
        trainset[index].append(disciplines[14].replace("\n", "").strip())
    elif len(disciplines) > 8:
        trainset[index].append(disciplines[9].replace("\n", "").replace('"', '').strip())
    else:
        print("Keine Disziplinen gefunden!", index, trainset[index])
        trainset[index].append("Keine Disziplinen gefunden!")
        errorlist.append(trainset[index])
        
    #print("\nline 2:", line)
    #print("trainset[i]:", trainset[i])
    #print("FERTIG")


start = time.time()
# Create two threads as follows
threads = []
for i in range(0,len(trainset)):
    if (i % 100 == 0):
                print("Schon wieder 100 Threads gestartet:", i)
    try:
        t = Thread(target = get_disciplines, args=(i, ))
        t.start()
        threads.append(t)
    except:
        print ("Error: unable to start thread")
        
for t in threads:
    #  join() stellt sicher, dass das Hauptprogramm wartet, bis alle Threads terminiert haben
    t.join()

print("Laufzeit in Minuten:", (time.time() - start) / 60)


Schon wieder 100 Threads gestartet: 0
Schon wieder 100 Threads gestartet: 100
Schon wieder 100 Threads gestartet: 200
Schon wieder 100 Threads gestartet: 300
Schon wieder 100 Threads gestartet: 400
Schon wieder 100 Threads gestartet: 500
Schon wieder 100 Threads gestartet: 600
Schon wieder 100 Threads gestartet: 700
Schon wieder 100 Threads gestartet: 800
Schon wieder 100 Threads gestartet: 900
Schon wieder 100 Threads gestartet: 1000
Schon wieder 100 Threads gestartet: 1100
Schon wieder 100 Threads gestartet: 1200
Schon wieder 100 Threads gestartet: 1300
Schon wieder 100 Threads gestartet: 1400
Schon wieder 100 Threads gestartet: 1500
Schon wieder 100 Threads gestartet: 1600
Schon wieder 100 Threads gestartet: 1700
Schon wieder 100 Threads gestartet: 1800
Schon wieder 100 Threads gestartet: 1900
Schon wieder 100 Threads gestartet: 2000
Schon wieder 100 Threads gestartet: 2100
Schon wieder 100 Threads gestartet: 2200
Schon wieder 100 Threads gestartet: 2300
Schon wieder 100 Threads ges

Exception in thread Thread-1649:
Traceback (most recent call last):
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-8e0ee8091118>", line 53, in get_disciplines
    for i, child in enumerate(morelinks.children):
AttributeError: 'NoneType' object has no attribute 'children'



Schon wieder 100 Threads gestartet: 10000
Schon wieder 100 Threads gestartet: 10100
Schon wieder 100 Threads gestartet: 10200
Kein Open-Edition-Link gefunden! 5217 ['musermeku_861', 'de', 'Angelika Schoder', '299', 'N/A', '2013-11-21', 'N/A', 'musermeku', '861', 'http://musermeku.hypotheses.org/861', 'Einblick in die Nachkriegszeit – Das AJR Journal']
Schon wieder 100 Threads gestartet: 10300
Kein Open-Edition-Link gefunden! 5258 ['zwopktnull_184', 'de', 'Christian Bunnenberg', '442', 'N/A', '2014-07-30', 'N/A', 'zwopktnull', '184', 'http://zwopktnull.hypotheses.org/184', 'Kaiserdom-App']
Schon wieder 100 Threads gestartet: 10400
Schon wieder 100 Threads gestartet: 10500
Schon wieder 100 Threads gestartet: 10600
Schon wieder 100 Threads gestartet: 10700
Schon wieder 100 Threads gestartet: 10800
Schon wieder 100 Threads gestartet: 10900
Kein Open-Edition-Link gefunden! 5672 ['musermeku_755', 'de', 'MusErMeKu Gast', '642', 'N/A', '2013-10-10', 'N/A', 'musermeku', '755', 'http://musermeku

Exception in thread Thread-8297:
Traceback (most recent call last):
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\http\client.py", line 1331, in getresponse
    response.begin()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\http\client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\http\client.py", line 258, in _read

Kein Open-Edition-Link gefunden! 8339 ['musermeku_1518', 'de', 'Angelika Schoder', '54', 'N/A', '2014-06-12', 'N/A', 'musermeku', '1518', 'http://musermeku.hypotheses.org/1518', 'George Clooney im Museum – Was Kulturinstitutionen vom Clickbait -Effekt lernen können']
Schon wieder 100 Threads gestartet: 14600
Schon wieder 100 Threads gestartet: 14700
Schon wieder 100 Threads gestartet: 14800
Kein Open-Edition-Link gefunden! 8554 ['musermeku_910', 'de', 'Angelika Schoder', '695', 'N/A', '2013-12-26', 'N/A', 'musermeku', '910', 'http://musermeku.hypotheses.org/910', 'Von der Doktorarbeit zum Buch in 11 Schritten – Teil 1']
Schon wieder 100 Threads gestartet: 14900
Schon wieder 100 Threads gestartet: 15000
Schon wieder 100 Threads gestartet: 15100
Schon wieder 100 Threads gestartet: 15200
Schon wieder 100 Threads gestartet: 15300
Schon wieder 100 Threads gestartet: 15400


...

Exception in thread Thread-21358:
Traceback (most recent call last):
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\http\client.py", line 1331, in getresponse
    response.begin()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\http\client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\http\client.py", line 266, in _rea

Kein Open-Edition-Link gefunden! 21386 ['musermeku_894', 'de', 'Angelika Schoder', '195', 'N/A', '2013-12-12', 'N/A', 'musermeku', '894', 'http://musermeku.hypotheses.org/894', 'Leseempfehlung: Aktuelle Rezensionen von Oliver Sukrow']


Exception in thread Thread-21361:
Traceback (most recent call last):
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\http\client.py", line 1331, in getresponse
    response.begin()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\http\client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\http\client.py", line 266, in _rea

Exception in thread Thread-18592:
Traceback (most recent call last):
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\http\client.py", line 1331, in getresponse
    response.begin()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\http\client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\http\client.py", line 266, in _rea

Exception in thread Thread-19225:
Traceback (most recent call last):
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\http\client.py", line 1331, in getresponse
    response.begin()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\http\client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "c:\users\hartmann\appdata\local\programs\python\python36\lib\http\client.py", line 266, in _rea

In [5]:
# show errors
print(len(errorlist))
print(errorlist)

0
[]


Speicherung der deutschen Blogbeiträge und ihrer Metadaten

In [6]:
# add ; subjects ; themes to de_labeled_metadata.csv
print(type(trainset))

trainset.sort()
np_lines = np.array(trainset)
#print(np_lines)

with open(folder+'/de_labeled_metadata.csv', 'w', newline='', encoding="utf-8") as labeledcsv:
    labeled = csv.writer(labeledcsv, delimiter = ";")
    labeled.writerow(["filename", "language", "author", "numwords", "category", "date", "licence", "blog", "post", "url", "title", "disciplines", "themes"])
    
    for i, line in enumerate(np_lines):
        labeled.writerow(line)
        

    



<class 'list'>


In [8]:
# move all german files to folder txt_de
newfolder = folder+'/txt_de'
if not os.path.exists(newfolder):
    os.makedirs(newfolder)


newfilelist = os.listdir(newfolder)
newfilelist.sort()
oldfolder = folder+'/txt'
filelist = os.listdir(oldfolder)
filelist.sort()
#print(trainset[0])
#print(len(trainset))
#trainset.sort()

for line in trainset:
    file = line[0] + '.txt'
    if (file in filelist) and (file not in newfilelist):
        shutil.copy2((oldfolder+'/'+file), (newfolder+'/'+file))
        #print("deutsch: ", (oldfolder+'/'+file))
    else:
        #print("Nicht deutsch")
        continue

In [9]:
# 100 missing files in folder 'txt'
missing = []
filelist = os.listdir(newfolder)
filelist.sort()
#trainset.sort()
for line in trainset:
    file = line[0] + '.txt'
    if file not in filelist:
        missing.append(file)
        #print("deutsch: ", (directory+'/'+file))
    else:
        #print("Nicht deutsch")
        continue
    
print(missing)
print(len(missing))

[]
0


In [10]:
# open german metadata file: de_labeled_metadata.csv
# and every Blogpost

filelist = os.listdir(newfolder)
filelist.sort()
lines = [] # alle in de_labeled_metadata.csv verzeichnete Blogposts
corpus = [] # deutschsprachige Blogposts
labels = [] # zugehörige Labels
errors = [] # in metadata.csv verzeichnete, aber in hypoposts-txt.zip nicht enthaltene Blogposts
filenames = [] # Blogposts ohne Fehler
onelabel = [] # Blogposts mit nur einer Art von Label (Thema oder Disziplin)

with open(folder+'/de_labeled_metadata.csv', 'r', encoding='utf-8') as openfile:
    metadata = openfile.readlines()
    openfile.close()
    #print(metadata[0])
    for i, line in enumerate(metadata[1:]):
        #print(i)
        lines.append(line.split(";"))
        #print("\nFile:", lines[i][0])
        #print("Themes:", lines[i][11])
        #print("Disciplines:", lines[i][12])
        file = (lines[i][0] + '.txt')
        #print("Filename:", file)
        
        if file in filelist:
            
            with open((newfolder+'/'+file), 'r', encoding='utf-8') as textfile:
                text = textfile.read()
                textfile.close()
                filenames.append(file)
                corpus.append(text)
              
            if len(lines[i]) > 12:
                labels.append(lines[i][11] + "; " + lines[i][12])
            elif len(lines[i]) > 10:
                labels.append(lines[i][11])
                onelabel.append(file)
            else:
                print("keine Disziplin gefunden!", lines[i])
                
        else:
            print("File nicht gefunden!", file)
            errors.append(file)
            continue

print("\n")
print(len(corpus))
print(len(labels))
print(len(filenames))
print(len(errors))
print(len(onelabel))

for blog in onelabel:
    print(blog)



21387
21387
21387
0
36
aes_1.txt
aes_105.txt
aes_126.txt
aes_128.txt
aes_13.txt
aes_138.txt
aes_165.txt
aes_182.txt
aes_198.txt
aes_204.txt
aes_207.txt
aes_214.txt
aes_228.txt
aes_232.txt
aes_240.txt
aes_245.txt
aes_257.txt
aes_26.txt
aes_273.txt
aes_288.txt
aes_29.txt
aes_315.txt
aes_317.txt
aes_322.txt
aes_327.txt
aes_329.txt
aes_334.txt
aes_337.txt
aes_349.txt
aes_365.txt
aes_37.txt
aes_47.txt
aes_51.txt
aes_80.txt
aes_88.txt
aes_99.txt


Erstellung der Datenbasis: Dateinamen, Blogbeiträge und zugehörige Klassen (Themen und Disziplinen)

In [11]:
# write csv-file de_labeled_corpus.csv: filename, classes, text

with open(folder+'/de_labeled_corpus.csv', 'w', newline='', encoding="utf-8") as labeledcsv:
    labeled = csv.writer(labeledcsv, delimiter = ";")
    labeled.writerow(["filename", "classes", "text"])
    
    for file, label, line in zip(filenames, labels, corpus):
        labeled.writerow([file.replace('\n', ' '), label.replace('\n', ''), line.replace('\n', ' ')])
        